In [1]:
using JuMP
using Gurobi

# Participants
people = ["Alice", "Bob", "Charlie", "Diane", "Ed", "Frank"]
balances = Dict(
    "Alice" => -15,
    "Bob" => 130,
    "Charlie" => -125,
    "Diane" => 150,
    "Ed" => 20,
    "Frank" => -160,
)

# Partition into debtors and creditors
debtors = [p for p in people if balances[p] < 0]
creditors = [p for p in people if balances[p] > 0]

# Big-M: upper bound on any possible transaction (sum of positives)
M = sum(abs(balances[p]) for p in people)

# Create model
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1)

# Decision variables
@variable(model, x[i in debtors, j in creditors] >= 0)       # Amount paid
@variable(model, y[i in debtors, j in creditors], Bin)       # 1 if payment made

# Objective: minimize number of payments
@objective(model, Min, sum(y[i, j] for i in debtors, j in creditors))

# Debt satisfaction: each debtor pays exactly what they owe
for i in debtors
    @constraint(model, sum(x[i, j] for j in creditors) == -balances[i])
end

# Credit satisfaction: each creditor receives exactly what they are owed
for j in creditors
    @constraint(model, sum(x[i, j] for i in debtors) == balances[j])
end

# Linking constraint
for i in debtors, j in creditors
    @constraint(model, x[i, j] <= M * y[i, j])
end

# Solve
optimize!(model)

# Print results
println("\nMinimum number of payments: ", objective_value(model))

for i in debtors, j in creditors
    amount = value(x[i, j])
    if amount > 1e-6
        println("$i pays $j: \$", round(amount, digits=2))
    end
end


Set parameter Username
Set parameter LicenseID to value 2688641
Academic license - for non-commercial use only - expires 2026-07-16
Set parameter OutputFlag to value 1
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 9 285, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 15 rows, 18 columns and 36 nonzeros
Model fingerprint: 0xaa1bbdfa
Variable types: 9 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 13 rows, 16 columns, 32 nonzeros
Variable types: 8 continuous, 8 integer (8 binary)
Found heuristic solution: objective 9.0000000
Found heuristic solution: objective 8.0000000
Found heuristi